In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from scipy import stats
# from fnn_datagenerator import BatchedFakeParticleDataset_All, flat_output_vars
from m5 import CustomKinematicNet, activation, hidden_layers, dfpath, modelsavepath, flat_output_vars, train_batch_size, prefix
from fnn_datagenerator import flat_output_vars, GeV_outputvars

import os
import sys
#os add to path /home/ddemler/HNLclassifier/DNN/
sys.path.append('/home/ddemler/HNLclassifier/DNN/')
from pytorch2python import create_data_loaders, create_test_loader
from tqdm.notebook import tqdm

sys.path.append('/home/ddemler/HNLclassifier/SignificancePlotting/')
sys.path.append('/home/ddemler/HNLclassifier/utils/')

from Significance_func import *





14 112


In [2]:
newmodelsavepath='/home/ddemler/HNLclassifier/saved_files/transfer_lmodel/Aug29/' + prefix + '_model.pt'

In [3]:
data_basepath='/home/ddemler/HNLclassifier/saved_files/extracted_data/TEST10_'
# TEST10_data_newAug28
fulldatapath=data_basepath + 'train_Aug29'
valdatapath=data_basepath + 'val_Aug29'
testdatapath=data_basepath + 'test_Aug29'

fulldata=pd.read_pickle(fulldatapath)
valdata=pd.read_pickle(valdatapath)

In [4]:
realdata_allvars = list(fulldata.columns)
print(realdata_allvars)


['event', 'weightOriginal', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'phi_1', 'phi_2', 'phi_3', 'phi_MET', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot', 'HNL

In [5]:

# old_model_input= ['1_eta', '1_mass', '1_phi', '1_pt', '2_eta?', '2_mass', '2_phi', '2_pt', '3_eta', '3_mass', '3_phi', '3_pt', 'MET_phi', 'MET_pt']
renamed_old_input_names=['eta_1', 'mass_1', 'phi_1', 'pt_1', 'eta_2', 'mass_2', 'phi_2', 'pt_2', 'eta_3', 'mass_3', 'phi_3', 'pt_3', 'phi_MET', 'pt_MET']
additionalinput_vars=['charge_1', 'charge_2', 'charge_3', 'channel','n_tauh', 'mass_hyp']
output=fulldata['signal_label']



def datamaker(data):
    outputdata_shape= (len(data['pt_MET']), len(flat_output_vars))
    inputdata_shape= (len(data['pt_MET']), len(renamed_old_input_names))

    inputdata=np.empty(inputdata_shape)
    outputdata=np.empty(outputdata_shape)

    for i, outvar in enumerate(flat_output_vars):
        if outvar in GeV_outputvars:
            outputdata[:,i]=data[outvar]/data['E_tot']
        else:
            outputdata[:,i]=data[flat_output_vars[i]]

    for i in range(len(renamed_old_input_names)):
        inputdata[:,i]=data[renamed_old_input_names[i]]

    input_tensor = torch.tensor(inputdata, dtype=torch.float32)
    output_tensor = torch.tensor(outputdata, dtype=torch.float32)

    return input_tensor, output_tensor

def additional_datamaker(data):
    outputdata_shape = (len(data['pt_MET']), 1)
    inputdata_shape = (len(data['pt_MET']), len(additionalinput_vars))

    inputdata = np.empty(inputdata_shape)
    outputdata = np.empty(outputdata_shape)

    for i, outvar in enumerate(additionalinput_vars):
        inputdata[:, i] = data[outvar]
    outputdata[:,0]=data['signal_label']

    input_tensor = torch.tensor(inputdata, dtype=torch.float32)
    output_tensor = torch.tensor(outputdata, dtype=torch.float32)

    return input_tensor, output_tensor
    
train_weight=fulldata['weightNorm'].to_numpy()
val_weight=valdata['weightNorm'].to_numpy()

print(type(train_weight))

train_weight_tensor=torch.tensor(train_weight, dtype=torch.float32)
val_weight_tensor=torch.tensor(val_weight, dtype=torch.float32)


train_inputdata, train_outputdata=datamaker(fulldata)
val_inputdata, val_outputdata=datamaker(valdata)

ad_train_inputdata, ad_train_outputdata=additional_datamaker(fulldata)
ad_val_inputdata, ad_val_outputdata=additional_datamaker(valdata)

train_inputfull=torch.cat((train_inputdata, ad_train_inputdata), dim=1)
val_inputfull=torch.cat((val_inputdata, ad_val_inputdata), dim=1)

print("inputdata shape: ", train_inputdata.shape)
print("outputdata shape: ", train_outputdata.shape)
print("val inputdata shape: ", val_inputdata.shape)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


<class 'numpy.ndarray'>
inputdata shape:  torch.Size([124208, 14])
outputdata shape:  torch.Size([124208, 112])
val inputdata shape:  torch.Size([24841, 14])


In [6]:
pretrained_model= CustomKinematicNet(input_size=14, hidden_layers=hidden_layers, lenoutput=len(flat_output_vars), activation_fn=activation)
pretrained_model.load_state_dict(torch.load(modelsavepath))
# pretrained_model.to(device)

<All keys matched successfully>

In [7]:
class FeatureExtractor(nn.Module):
    def __init__(self, pretrained_model):
        super(FeatureExtractor, self).__init__()
        self.pretrained = pretrained_model
        self.pretrained.layers = self.pretrained.layers[:-1]  # Remove the last layer

    def forward(self, x):
        return self.pretrained(x)


feature_extractor = FeatureExtractor(pretrained_model=pretrained_model)




In [8]:
class TransferCustomKinematicNet(nn.Module):
    def __init__(self, feature_extractor, additional_input_size, new_hidden_layers):
        super(TransferCustomKinematicNet, self).__init__()
        
        self.feature_extractor = feature_extractor
        
        # Freeze the pre-trained layers
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        # Define new hidden layers
        input_size = feature_extractor.pretrained.layers[-1].out_features + additional_input_size
        layer_sizes = [input_size] + new_hidden_layers + [1]  # Final output size is 1 for binary classification
        
        self.new_layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.new_layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
        
        self.activation_fn = nn.ReLU()
        self.output_activation = nn.Sigmoid()

    def forward(self, x, additional_input):
        features = self.feature_extractor(x)
        combined_input = torch.cat((features, additional_input), dim=-1)
        
        out = combined_input
        for i in range(len(self.new_layers) - 1):
            out = self.activation_fn(self.new_layers[i](out))
        
        out = self.output_activation(self.new_layers[-1](out))
        return out



In [9]:
train_dataset = torch.utils.data.TensorDataset(train_inputfull, ad_train_outputdata, train_weight_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=320, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(val_inputfull, ad_val_outputdata, val_weight_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=320, shuffle=False)

# Initialize the feature extractor with the pre-trained model
feature_extractor = FeatureExtractor(pretrained_model=pretrained_model)

# Initialize the new model
# Assuming YOUR_ADDITIONAL_INPUT_SIZE is the number of additional features you have
new_model = TransferCustomKinematicNet(feature_extractor, additional_input_size=len(additionalinput_vars), new_hidden_layers=[128])
new_model.to(device)


TransferCustomKinematicNet(
  (feature_extractor): FeatureExtractor(
    (pretrained): CustomKinematicNet(
      (layers): ModuleList(
        (0): Linear(in_features=14, out_features=512, bias=True)
        (1-2): 2 x Linear(in_features=512, out_features=512, bias=True)
        (3): Linear(in_features=526, out_features=512, bias=True)
        (4-5): 2 x Linear(in_features=512, out_features=512, bias=True)
        (6): Linear(in_features=526, out_features=512, bias=True)
        (7-8): 2 x Linear(in_features=512, out_features=512, bias=True)
        (9): Linear(in_features=526, out_features=512, bias=True)
        (10-11): 2 x Linear(in_features=512, out_features=512, bias=True)
        (12): Linear(in_features=526, out_features=512, bias=True)
        (13-14): 2 x Linear(in_features=512, out_features=512, bias=True)
        (15): Linear(in_features=526, out_features=512, bias=True)
        (16-17): 2 x Linear(in_features=512, out_features=512, bias=True)
        (18): Linear(in_featur

In [11]:
new_optimizer = torch.optim.AdamW(new_model.parameters(), lr=4e-3, weight_decay=1e-2)
new_criterion = nn.BCELoss()
new_scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(new_optimizer, 'min', patience=5, factor=0.3, verbose=True)
numepochs=100


# Binary Cross Entropy Loss for classification
criterion = nn.BCELoss()

# Training loop
epochs = 100
model_patience=15
best_valloss= np.inf
for epoch in range(epochs):
    new_model.train()
    weightedfull_trainloss=0
    for i, (combined_input, target, sample_weight) in enumerate(train_dataloader):
        # Split the combined input into input for the pre-trained model and additional input
        pretrained_input = combined_input[:, :train_inputdata.shape[1]].to(device)
        additional_input = combined_input[:, train_inputdata.shape[1]:].to(device)
        target = target.to(device)

        new_optimizer.zero_grad()
        
        # Forward pass
        outputs = new_model(pretrained_input, additional_input)
        
        # Compute Loss without reduction
        loss_unreduced = criterion(outputs, target)
        
        # Apply sample weights and compute the final loss
        weighted_loss = (loss_unreduced * sample_weight.to(device)).mean()

        weightedfull_trainloss += weighted_loss.item()
        
        # Backward pass and optimization
        weighted_loss.backward()
        new_optimizer.step()
    
    weightedfull_trainloss /= len(train_dataloader.dataset)
    new_model.eval()
    with torch.no_grad():
        val_loss = 0
        for i, (combined_input, target, sample_weight) in enumerate(val_dataloader):
            # Split the combined input into input for the pre-trained model and additional input
            pretrained_input = combined_input[:, :train_inputdata.shape[1]].to(device)
            additional_input = combined_input[:, train_inputdata.shape[1]:].to(device)
            target = target.to(device)

            # Forward pass
            outputs = new_model(pretrained_input, additional_input)
            
            # Compute Loss without reduction
            loss_unreduced = criterion(outputs, target)
            
            # Apply sample weights and compute the final loss
            weighted_loss = (loss_unreduced * sample_weight.to(device)).sum()
            
            val_loss += weighted_loss.item()
        
        val_loss /= len(val_dataloader.dataset)
        
        if val_loss < best_valloss:
            patience=model_patience
            best_valloss = val_loss
            epoch_best=epoch
            torch.save(new_model.state_dict(), newmodelsavepath)
        else:
            patience-=1
            if patience==0:
                print("Early stopping, saving epoch ", epoch_best)
                break
        
        new_scheduler.step(val_loss)
    

    print(f'Epoch {epoch + 1}, Train Loss: {weightedfull_trainloss:.4e}, Val Loss: {val_loss:.4e}')

    # You can add code here for validation and learning rate scheduling



(tensor([], device='cuda:1', dtype=torch.int64), tensor([], device='cuda:1', dtype=torch.int64))


KeyboardInterrupt: 

In [13]:
for i, (combined_input, target, sample_weight) in enumerate(train_dataloader):
    # print("combined input shape: ", combined_input.shape)
    for j in range(combined_input.shape[1]):
        print("Mean of column ", j, ":", combined_input[:, j].mean())
    # print("Combined Input: ", combined_input[:3, :])
    # print("Target: ", target)
    # print("Sample Weight: ", sample_weight)
    if i > 2:
        break

Mean of column  0 : tensor(-0.1080)
Mean of column  1 : tensor(0.7017)
Mean of column  2 : tensor(0.0949)
Mean of column  3 : tensor(84.2402)
Mean of column  4 : tensor(0.0102)
Mean of column  5 : tensor(0.6764)
Mean of column  6 : tensor(-0.1918)
Mean of column  7 : tensor(72.1675)
Mean of column  8 : tensor(-0.0113)
Mean of column  9 : tensor(0.1057)
Mean of column  10 : tensor(0.0620)
Mean of column  11 : tensor(79.3182)
Mean of column  12 : tensor(0.0453)
Mean of column  13 : tensor(98.1460)
Mean of column  14 : tensor(0.0312)
Mean of column  15 : tensor(0.0188)
Mean of column  16 : tensor(0.0250)
Mean of column  17 : tensor(0.)
Mean of column  18 : tensor(1.8687)
Mean of column  19 : tensor(447.3125)
Mean of column  0 : tensor(0.0051)
Mean of column  1 : tensor(0.7156)
Mean of column  2 : tensor(0.0248)
Mean of column  3 : tensor(96.7213)
Mean of column  4 : tensor(0.0710)
Mean of column  5 : tensor(0.7186)
Mean of column  6 : tensor(0.0675)
Mean of column  7 : tensor(70.7257)
Mea

In [ ]:
testdata=pd.read_pickle(testdatapath)
test_inputdata, test_outputdata=datamaker(testdata)
ad_test_inputdata, ad_test_outputdata=additional_datamaker(testdata)
test_inputfull=torch.cat((test_inputdata, ad_test_inputdata), dim=1)

test_weight=testdata['weightNorm'].to_numpy()
test_weight_tensor=torch.tensor(test_weight, dtype=torch.float32)

test_dataset = torch.utils.data.TensorDataset(test_inputfull, ad_test_outputdata, test_weight_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=None, shuffle=False)

new_model.load_state_dict(torch.load(newmodelsavepath))